# **Basic Machine Learning Pipeline for Binary classification**

**Dataset used:** Customised APS Failure at Scania Truck

**Major Challenges:** Missing Data, Class Imbalance

**Models used:** Logistic Regression, SVM, RandomForest, XGBoost


### **Logistic Regression**


In [ ]:
#Libraries
import numpy as np
import pandas as pd

#Preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV 
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

#Splitting
from sklearn.model_selection import train_test_split

#Training Model
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [ ]:
#costMetric
def cost(y_test,y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    #return (100*fp/(tn+fp+fn+tp) + 3500*fn/(tn+fp+fn+tp))
    
    return (100*fp + 3500*fn)

cost_score = make_scorer((cost), greater_is_better=False)

In [ ]:
#reading dataset using pandas
df = pd.read_csv('aps_failure_train.csv')
y = df['class']
X = df.drop('class', axis = 1)
#Test_data
df_test = pd.read_csv('aps_failure_test.csv')
x_test = df_test.drop('class', axis = 1)
X.shape

(57000, 170)

In [ ]:
imp = SimpleImputer(missing_values= np.nan, strategy='mean')
X = imp.fit_transform(X)
x_test = imp.transform(x_test)

In [ ]:
Scaler = StandardScaler()
X = Scaler.fit_transform(X)
x_test=Scaler.transform(x_test)

In [ ]:
pca = PCA(n_components = 100)
X = pca.fit_transform(X);
x_test = pca.transform(x_test)

In [ ]:
#Splitting Dataset in stratified training and test data
X_train,  X_val,y_train, y_val = train_test_split(X,y,test_size = 0.2, stratify=y)

In [ ]:
Logreg = LogisticRegression(random_state=9, solver='newton-cg', C = 0.1, penalty = 'l2')
Logreg.fit(X_train,y_train)
#cross_val_score(Logreg, X_train,y_train,scoring = cost_score,cv=10,verbose=3)
y_pred = Logreg.predict(X_val)
print(confusion_matrix(y_val,y_pred))

[[11157    37]
 [   80   126]]


In [ ]:
#Used newton-cg because lbfgs did not converge. IT is similar to newton-cg which is more expensive due to computation of 
#inverse of Hessian Matrix. In contrast, lbfgs uses approximated inverse but fails to converse. 
logreg_train = LogisticRegression(random_state=8, solver='newton-cg', C = 0.1, penalty = 'l2')
logreg_train.fit(X,y)
y_pred_fin = logreg_train.predict(x_test)
np.size(y_pred_fin)

19000

In [ ]:
ID = np.linspace(1,19000,19000)
pd.DataFrame(y_pred_fin,ID, columns=['predictions']).to_csv('prediction.csv')

### **Support Vector Machine**



In [ ]:
#Regular old
import numpy as np
import pandas as pd

#Preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV 
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix

from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

#Splitting
from sklearn.model_selection import train_test_split

#Training Model
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

#Upsampling
from imblearn.over_sampling import SMOTE 
import matplotlib.pyplot as plt
from collections import Counter

In [ ]:
#costMetric
def cost(y_test,y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    #return (100*fp/(tn+fp+fn+tp) + 3500*fn/(tn+fp+fn+tp))
    
    return (100*fp + 3500*fn)

cost_score = make_scorer((cost), greater_is_better=False)

In [ ]:
#reading dataset using pandas
df = pd.read_csv('aps_failure_train.csv')
y = df['class']
X = df.drop('class', axis = 1)
#Test_data
df_test = pd.read_csv('aps_failure_test.csv')
x_test = df_test.drop('class', axis = 1)
X.shape

(57000, 170)

In [ ]:
imp = SimpleImputer(missing_values= np.nan, strategy='mean')
X_imp = imp.fit_transform(X)
x_test_imp = imp.transform(x_test)

In [ ]:
Scaler = StandardScaler()
X_scal = Scaler.fit_transform(X_imp)
x_test_scal=Scaler.transform(x_test_imp)

In [ ]:
pca = PCA(n_components = 100)
X_pca = pca.fit_transform(X_scal);
x_test_pca = pca.transform(x_test_scal)

In [ ]:
print('Original dataset shape %s' % Counter(y))
sm = SMOTE(random_state=42)
X_b, y_b = sm.fit_resample(X_pca, y)
print('Resampled dataset shape %s' % Counter(y_b))
print(np.size(y))

Original dataset shape Counter({False: 55968, True: 1032})
Resampled dataset shape Counter({False: 55968, True: 55968})
57000


In [ ]:
#Splitting Dataset in stratified training and test data
X_train,  X_val,y_train, y_val = train_test_split(X_b,y_b,test_size = 0.2, stratify=y_b)

In [ ]:
#SupportVectorMachine
svm = SVC()
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10],
 'gamma': [0.001, 0.01, 0.1, 1],
    'kernel': ['rbf', 'linear']}
Rf_grid = GridSearchCV(svm, param_grid, verbose = 3,scoring = cost_score,cv=5)
#cross_val_score(Rf, X_train,y_train,scoring = cost_score,cv=10,verbose=3)
Rf_grid.fit(X_train,y_train)
Rf_grid.best_params_

In [ ]:
#SVM
svm= SVC(gamma = 0.1, C=10, verbose = 3,kernel = 'rbf')
svm.fit(X_train,y_train)
y_pred_fin = svm.predict(X_val)

[LibSVM]

In [ ]:
print(confusion_matrix(y_val,y_pred_fin))

[[10341   853] 
 [   91   11103]]


In [ ]:
svm_fin= SVC(gamma = 0.1, C=10, verbose = 3,kernel = 'rbf')
svm_fin.fit(X_b,y_b)

In [ ]:
y_pred_fin = svm_fin.predict(x_test_pca)
ID = np.linspace(1,19000,19000)
pd.DataFrame(y_pred_fin,ID, columns=['predictions']).to_csv('prediction.csv')

### **Random Forest**



In [ ]:
#Regular old
import numpy as np
import pandas as pd

#Preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV 
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix

from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

#Splitting
from sklearn.model_selection import train_test_split

#Training Model
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

#Upsampling
from imblearn.over_sampling import SMOTE 
import matplotlib.pyplot as plt
from collections import Counter

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [ ]:
#costMetric
def cost(y_test,y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    
    return (100*fp + 3500*fn)

cost_score = make_scorer((cost), greater_is_better=False)

In [ ]:
#reading dataset using pandas
df = pd.read_csv('aps_failure_train.csv')
y = df['class']
X = df.drop('class', axis = 1)
#Test_data
df_test = pd.read_csv('aps_failure_test.csv')
x_test = df_test.drop('class', axis = 1)
X.shape

(57000, 170)

In [ ]:
imp = SimpleImputer(missing_values= np.nan, strategy='mean')
X_imp = imp.fit_transform(X)
x_test_imp = imp.transform(x_test)

In [ ]:
Scaler = StandardScaler()
X_scal = Scaler.fit_transform(X_imp)
x_test_scal=Scaler.transform(x_test_imp)

In [ ]:
pca = PCA(n_components = 100)
X_pca = pca.fit_transform(X_scal);
x_test_pca = pca.transform(x_test_scal)
#explains 97.74 percent of the data
mp = abs( pca.components_ )
np.size(mp)
np.sum(pca.explained_variance_ratio_)

0.9775041677717252

In [ ]:
print('Original dataset shape %s' % Counter(y))
sm = SMOTE(random_state=42)
X_b, y_b = sm.fit_resample(X_scal, y)
print('Resampled dataset shape %s' % Counter(y_b))
print(np.size(y))

Original dataset shape Counter({False: 55968, True: 1032})
Resampled dataset shape Counter({False: 55968, True: 55968})
57000


In [ ]:
disc = KBinsDiscretizer(n_bins=7, encode='onehot', 
                        strategy='uniform')
X_kb = disc.fit_transform(X_b)
x_test_kb = disc.transform(x_test_scal)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_discretization.py:159: UserWarning: Feature 89 is constant and will be replaced with 0.
  "replaced with 0." % jj)


In [ ]:
#Splitting Dataset in stratified training and test data
X_train,  X_val,y_train, y_val = train_test_split(X_kb,y_b,test_size = 0.2, stratify=y_b)

In [ ]:
#Random Forest
Rf = RandomForestClassifier()
param_grid = {'max_depth': [50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
'n_estimators':[50,100]}
Rf_grid = GridSearchCV(Rf, param_grid, verbose = 3,scoring = cost_score,cv=5)
#cross_val_score(Rf, X_train,y_train,scoring = cost_score,cv=10,verbose=3)
Rf_grid.fit(X_train,y_train)

In [ ]:
Rf_grid.best_params_

In [ ]:
rf_train = RandomForestClassifier(n_estimators=50, verbose = 3,min_samples_leaf =1, min_samples_split =2, max_depth = None)
rf_train.fit(X_train,y_train)
y_pred_fin = rf_train.predict(X_val)
print(confusion_matrix(y_val,y_pred_fin))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 50


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.0s remaining:    0.0s


building tree 2 of 50


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    4.1s remaining:    0.0s


building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 o

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  1.8min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[[10843   351]
 [  245 10949]]


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.5s finished


In [ ]:
rf_final = RandomForestClassifier(n_estimators=50, verbose = 3,min_samples_leaf =1, min_samples_split =2, max_depth = None)
rf_final.fit(X_b,y_b)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 50


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.7s remaining:    0.0s


building tree 2 of 50


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.2s remaining:    0.0s


building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 o

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  1.4min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=3, warm_start=False)

In [ ]:
y_pred_fin = rf_final.predict(x_test_scal)
ID = np.linspace(1,19000,19000)
pd.DataFrame(y_pred_fin,ID, columns=['predictions']).to_csv('prediction.csv')

### **XGBoost**

In [ ]:
#Regular old
import numpy as np
import pandas as pd

#Preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV 
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix

from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

#Splitting
from sklearn.model_selection import train_test_split

#Upsampling
from imblearn.over_sampling import SMOTE 
import matplotlib.pyplot as plt
from collections import Counter

#model
import xgboost as xgb

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [ ]:
#costMetric
def cost(y_test,y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    #return (100*fp/(tn+fp+fn+tp) + 3500*fn/(tn+fp+fn+tp))
    
    return (100*fp + 3500*fn)

cost_score = make_scorer((cost), greater_is_better=False)

In [ ]:
#reading dataset using pandas
df = pd.read_csv('aps_failure_train.csv')
y = df['class']
X = df.drop('class', axis = 1)
#Test_data
df_test = pd.read_csv('aps_failure_test.csv')
x_test = df_test.drop('class', axis = 1)
X.shape

(57000, 170)

In [ ]:
data_dmatrix = xgb.DMatrix(data=X,label=y)
test_dmatrix =  xgb.DMatrix(data=x_test)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [ ]:
X_train,  X_val,y_train, y_val = train_test_split(X,y,test_size = 0.2, stratify=y)

In [ ]:
xg_cl = xgb.XGBClassifier()
param_grid = {'n_estimators': [30,50,100,150,200],
 'max_depth': [5,10,20,30],
    'learning_rate': [0.001]}
Rf_grid = GridSearchCV(xg_cl, param_grid, verbose = 3,scoring = cost_score,cv=5)
Rf_grid.fit(X_train,y_train)
Rf_grid.best_params

In [ ]:
#cross_val_score(xg_reg, X_train,y_train, cv = 5, verbose=3, scoring = cost_score)
xg_reg = xgb.XGBClassifier(n_estimators=100, learning_rate=0.01, gamma= 10,min_child_weight = 10,subsample=0.5,scale_pos_weight = 55)
xg_reg.fit(X_train,y_train,eval_metric=cost_score)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=10,
              learning_rate=0.01, max_delta_step=0, max_depth=3,
              min_child_weight=10, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=55, seed=None,
              silent=None, subsample=0.5, verbosity=1)

In [ ]:
preds = xg_reg.predict(X_val)
print(confusion_matrix(y_val, preds))

[[10640   554]
 [    8   198]]


In [ ]:
xg_fin = xgb.XGBClassifier(n_estimators=100, learning_rate=0.01, gamma= 10,min_child_weight = 10,subsample=0.5,scale_pos_weight = 55)
xg_fin.fit(X,y, eval_metric= cost_score)

In [ ]:
y_pred_fin = xg_fin.predict(x_test)
ID = np.linspace(1,19000,19000)
pd.DataFrame(y_pred_fin,ID, columns=['predictions']).to_csv('prediction1.csv')


With different set of parameters:

*Score:*

*Public Leaderboard: -5.57894*

*Private Leaderboard: -8.58947*

In [ ]:
#cross_val_score(xg_reg, X_train,y_train, cv = 5, verbose=3, scoring = cost_score)
xg_reg = xgb.XGBClassifier(n_estimators=30,learning_rate=0.3,gamma = 20,subsample=0.5,scale_pos_weight =55,max_delta_step=10,seed=42)
xg_reg.fit(X_train,y_train,eval_metric=cost_score)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=20,
              learning_rate=0.3, max_delta_step=10, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=30, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=55, seed=42,
              silent=None, subsample=0.5, verbosity=1)

In [ ]:
preds = xg_reg.predict(X_val)
preds

array([False, False, False, ..., False, False, False])

In [ ]:
print(confusion_matrix(y_val, preds))

[[10896   298]
 [    8   198]]


In [ ]:
xg_fin = xgb.XGBClassifier(n_estimators=30,learning_rate=0.3,gamma = 20,subsample=0.5,scale_pos_weight =55,max_delta_step=10,seed=42)
xg_fin.fit(X,y, eval_metric= cost_score)

In [ ]:
y_pred_fin = xg_fin.predict(x_test)
ID = np.linspace(1,19000,19000)
pd.DataFrame(y_pred_fin,ID, columns=['predictions']).to_csv('prediction1.csv')

With different set of parameters:
Decreasing the learning rate and increase max_delta to encourage more conservative learning. 

*Score:*

*Public Leaderboard: -4.86315

*Private Leaderboard: -6.33684

In [ ]:
#cross_val_score(xg_reg, X_train,y_train, cv = 5, verbose=3, scoring = cost_score)
xg_reg = xgb.XGBClassifier(n_estimators=120,learning_rate=0.1,gamma = 10,subsample=0.5,scale_pos_weight =55,max_delta_step=10)
xg_reg.fit(X_train,y_train,eval_metric=cost_score)
preds = xg_reg.predict(X_val)
print(confusion_matrix(y_val, preds))

[[10924   270]
 [    9   197]]


In [ ]:
xg_fin = xgb.XGBClassifier(n_estimators=120,learning_rate=0.1,gamma = 10,subsample=0.5,scale_pos_weight =55,max_delta_step=10)
xg_fin.fit(X,y, eval_metric= cost_score)
y_pred_fin = xg_fin.predict(x_test)
ID = np.linspace(1,19000,19000)
pd.DataFrame(y_pred_fin,ID, columns=['predictions']).to_csv('prediction1.csv')